# 11. Plutonian Pebbles

You find stones that break physics. Based on what stone is engraved on them, each time you blink a stone does one of a following:
- If the stone is engraved with the stone 0, it is replaced by a stone engraved with the number 1.
- If the stone is engraved with a number that has an even number of digits, it is replaced by two stones. The left half of the digits are engraved on the new left stone, and the right half of the digits are engraved on the new right stone. (The new numbers don't keep extra leading zeroes: 1000 would become stones 10 and 0.)
- If none of the other rules apply, the stone is replaced by a new stone; the old stone's number multiplied by 2024 is engraved on the new stone.

How many stones do you have after blinking 25 tmes?

In [5]:
import numpy as np

def blink(stones):
    stonesAfterBlink = []
    for stone in stones:
        if stone == 0:
            stonesAfterBlink.append(1)            
        elif len(str(stone)) % 2 == 0:
            mid = len(str(stone)) // 2
            stonesAfterBlink.append(int(str(stone)[:mid])) # Left stone
            rightHalf = str(stone)[mid:].lstrip("0")
            if rightHalf == "":
                rightHalf = "0"
            stonesAfterBlink.append(int(rightHalf)) # Right stone without leading zeroes
        else:
            stonesAfterBlink.append((stone * 2024) % (2**63 - 1))  # Using a large prime number as modulus
    
    return stonesAfterBlink

InitStones = np.loadtxt("input.txt", dtype=int)

stoneHistory = [InitStones]
for i in range(25):
    # print(stoneHistory[-1])
    stoneHistory.append(blink(stoneHistory[-1]))

print("Part 1: ", len(stoneHistory[-1]))

Part 1:  183646


C:\Users\cenet\AppData\Local\Temp\ipykernel_22508\3254666136.py:16: RuntimeWarning: overflow encountered in scalar multiply
  stonesAfterBlink.append((stone * 2024) % (2**63 - 1))  # Using a large prime number as modulus


I think the problem is that number gets sooo big python cant handle it!

I browsed some memes on r/adventofcode and figured out that numbers eventually start repeating thamselves.
Lets just obseerve changes to `17`

`17 -> 1 7 -> 2024 14168`

`2024 -> 20`

`20 -> 2 0`    `0 -> 1`

`2 -> 4048`

`4048 -> 2024`

In [4]:
class Stone():
    def __init__(self, number):
        stonesAfterBlink = []

        if number == 0:
            stonesAfterBlink.append(1)            
        elif len(str(number)) % 2 == 0:
            mid = len(str(number)) // 2
            stonesAfterBlink.append(int(str(number)[:mid])) # Left stone
            rightHalf = str(number)[mid:].lstrip("0")
            if rightHalf == "":
                rightHalf = "0"
            stonesAfterBlink.append(int(rightHalf)) # Right stone without leading zeroes
        else:
            stonesAfterBlink.append(number * 2024)  # Using a large prime number as modulus

        self.id = number
        self.After = stonesAfterBlink


StonesInLine = {125: 1, 17: 1}


numOfStones = sum(StonesInLine.values())

for i in range(6):
    newStonesInLine = {}

    for stone in StonesInLine.copy():
        resultingstones = Stone(stone).After
        for y in resultingstones:
                newStonesInLine[y] = StonesInLine[stone]

    StonesInLine = newStonesInLine
    numOfStones += sum(StonesInLine.values())

    print(f"{i+1} {sum(StonesInLine.values()):7} {StonesInLine}")

print()
print("Part 1: ", numOfStones)
print(StonesInLine)

1       3 {253000: 1, 1: 1, 7: 1}
2       4 {253: 1, 0: 1, 2024: 1, 14168: 1}
3       5 {512072: 1, 1: 1, 20: 1, 24: 1, 28676032: 1}
4       8 {512: 1, 72: 1, 2024: 1, 2: 1, 0: 1, 4: 1, 2867: 1, 6032: 1}
5      12 {1036288: 1, 7: 1, 2: 1, 20: 1, 24: 1, 4048: 1, 1: 1, 8096: 1, 28: 1, 67: 1, 60: 1, 32: 1}
6      15 {2097446912: 1, 14168: 1, 4048: 1, 2: 1, 0: 1, 4: 1, 40: 1, 48: 1, 2024: 1, 80: 1, 96: 1, 8: 1, 6: 1, 7: 1, 3: 1}

Part 1:  49
{2097446912: 1, 14168: 1, 4048: 1, 2: 1, 0: 1, 4: 1, 40: 1, 48: 1, 2024: 1, 80: 1, 96: 1, 8: 1, 6: 1, 7: 1, 3: 1}


Okay Im clearly doing something wrong...

I browsed the sum a bit more and found that recursive solution with `@cache` should be used!

In [2]:
from functools import cache


@cache # Without this, function will always compute already computed stuff, with this, one it finds loops, it will just take the result from cache
def CountStones(stone, blink = 6):
    if blink == 0:
        return 1 # stone wont multiply on last blink
    if stone == 0:
        return CountStones(1, blink - 1) # If you find stone 0, it becomes stone 1

    elif len(str(stone)) % 2 == 0:
        mid = len(str(stone)) // 2
        rightHalf = str(stone)[mid:].lstrip("0")
        if rightHalf == "":
            rightHalf = "0"
        return (CountStones(int(str(stone)[:mid]), blink - 1) + CountStones(int(rightHalf), blink - 1))
    else:
        return CountStones(stone * 2024, blink - 1) # Using a large prime number as modulus


data = [int(i) for i in open("input.txt", "r").read().split(" ")]
print("Part 1: ", sum(map(lambda x: CountStones(x, blink = 25), data)))
print("Part 2: ", sum(map(lambda x: CountStones(x, blink = 75), data)))

Part 1:  218079
Part 2:  259755538429618


`@cache` basically stores what function had already computed and once it gets same input parameters, it just takes result from cache, reducing time takes to compute by orders of magnitude